In [1]:
!pip install pandas requests



[notice] A new release of pip is available: 24.0 -> 24.1.1
[notice] To update, run: pip install --upgrade pip


In [16]:
import pandas as pd
import requests
import time

# Load the CSV file
file_path = 'top_issuer.csv'
df = pd.read_csv(file_path)
df['Issuer bin'] = df['Issuer bin'].astype(int)

# Function to get bank details from Binlist API
def get_bank_details(bin_number):
    url = f"https://lookup.binlist.net/{bin_number}"
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        bank_name = data.get("bank", {}).get("name", "Unknown")
        print(f"BIN: {bin_number}, Bank: {bank_name}")
        return bank_name
    elif response.status_code == 429:
        print("Error 429: Too Many Requests. Stopping process.")
        return "Error 429"
    else:
        print(f"Error {response.status_code}: {response.reason}")
        return "Unknown"

# Create a dictionary to map BIN to bank name
bin_to_bank = {}
for bin_number in df['Issuer bin']:
    if not pd.isna(bin_number):
        bank_name = get_bank_details(int(bin_number))
        if bank_name == "Error 429":
            break
        bin_to_bank[bin_number] = bank_name

# Print the map of BIN to bank name
for bin_number, bank_name in bin_to_bank.items():
    print(f"BIN: {bin_number}, Bank: {bank_name}")

# Optional: Save the results to a new CSV file
result_df = pd.DataFrame(list(bin_to_bank.items()), columns=['Issuer bin', 'Bank'])
result_df.to_csv('output/Issuer_bin_to_bank.csv', index=False)


Error 429: Too Many Requests. Stopping process.


In [11]:
result_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 394 entries, 0 to 393
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Issuer bin  394 non-null    int64 
 1   Bank        394 non-null    object
dtypes: int64(1), object(1)
memory usage: 6.3+ KB


In [4]:
result_df[result_df["Bank"] == "Unknown"]

,Issuer bin,Bank
4,498011.0,Unknown
5,453725.0,Unknown
6,498001.0,Unknown
7,498003.0,Unknown
8,498009.0,Unknown
...,...,...
389,45412299.0,Unknown
390,454213.0,Unknown
391,45412255.0,Unknown
392,453429.0,Unknown


In [9]:

df.head()

,Issuer bin,count
0,470881,7930
1,498000,7639
2,498005,6876
3,429769,4443
4,498011,1838


In [5]:
result_df.shape

(394, 2)

In [6]:
# Optional: Save the results to a new CSV file
result_df = pd.DataFrame(list(bin_to_bank.items()), columns=['Issuer bin', 'Bank'])
result_df.to_csv('output/Issuer_bin_to_bank.csv', index=False)